# CircuitSEQ API examples

In [1]:
import requests
import os
import pandas as pd

In [2]:
rest_api_url = "https://circuitseq.iwr.uni-heidelberg.de:8080"

## Public API

This can be accessed by anyone without requiring an authorization token.

### `/remaining`

- returns how many samples are still available this week
- doesn't require an authentication token

In [3]:
# requests is the standard Python library for this:
response = requests.get(f"{rest_api_url}/remaining")

In [4]:
# status code is 200 on success, 401 or other if something went wrong:
response.status_code

200

In [5]:
# the request returns data in json format (which is converted to a Python dict by requests):
response.json()

{'remaining': 96}

## Admin API

An authorization token for an admin account is required

In [6]:
# if we try to access an admin endpoint without suitable authentication we get an error:
response = requests.get(f"{rest_api_url}/admin/samples")

In [7]:
response.status_code

401

In [8]:
response.json()

{'msg': 'Missing Authorization Header'}

## Auth token
- you can get create an API token from the admin page
- then export it as an evironment variable on your computer e.g. `export CIRCUITSEQ_API_TOKEN=abca12421...`
- need to add this token to your header when making requests if you want to authenticate yourself
- NOTE: currently these tokens are invalidated whenever the code is updated & the server is restarted (which is currently quite often!)
  - if this happends your previously working authorized request will return a 422 status code with `Signature verification failed` in the json
  - generating a new token from the admin page and exporting it should fix this

In [9]:
# get the API token from the environment variable (better not to directly add it to your script)
auth_token = os.environ["CIRCUITSEQ_API_TOKEN"]
# make an authorization header for requests to use
auth_header = {"Authorization": f"Bearer {auth_token}"}

### `/admin/samples`

- returns current samples and previous samples

In [21]:
response = requests.get(f"{rest_api_url}/admin/samples", headers=auth_header)

In [22]:
response.status_code

200

In [23]:
response.json().keys()

dict_keys(['current_samples', 'previous_samples'])

In [24]:
pd.DataFrame(response.json()["previous_samples"])

,date,email,has_results_fasta,has_results_gbk,has_results_zip,id,name,primary_key,reference_sequence_description,running_option
0,"Thu, 17 Nov 2022 00:00:00 GMT",user@embl.de,False,False,False,4,ZIP_TEST_pDONR_amilCP,22_46_A4,None,dna_r9.4.1_450bps_sup.cfg
1,"Wed, 16 Nov 2022 00:00:00 GMT",user@embl.de,True,True,True,3,ZIP_TEST_pCW571,22_46_A3,None,dna_r9.4.1_450bps_sup.cfg
2,"Tue, 15 Nov 2022 00:00:00 GMT",user@embl.de,False,False,False,2,ZIP_TEST_pMC_Final_Kan,22_46_A2,None,dna_r9.4.1_450bps_sup.cfg
3,"Mon, 14 Nov 2022 00:00:00 GMT",user@embl.de,False,False,False,1,no_ref_seq,22_46_A1,None,dna_r9.4.1_450bps_sup.cfg


### `/admin/zipsamples`

- download a zipfile with a tsv of the samples table and reference sequence fasta files for this week

In [25]:
response = requests.post(f"{rest_api_url}/admin/zipsamples", headers=auth_header)

In [26]:
response.status_code

200

In [27]:
# write downloaded binary data to a file:
with open("samples.zip", "wb") as f:
    f.write(response.content)

### `/admin/result`

- upload a zipfile of results for a sample
- assumes the zipfile is named `[PRIMARY_KEY]_[NAME].zip`
- on success returns the path on the server where the zipfile is stored

In [28]:
with open(
    "../backend/tests/data/results/22_46_A2_ZIP_TEST_pMC_Final_Kan.zip", "rb"
) as f:
    response = requests.post(
        f"{rest_api_url}/admin/result", files={"file": f}, headers=auth_header
    )

In [29]:
response.status_code

200

In [31]:
response.json()

{'message': '/circuit_seq_data/2022/46/results/22_46_A2_ZIP_TEST_pMC_Final_Kan.zip'}